In [ ]:
# Don't import any other libraries
from collections import defaultdict
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data
import torchtext 
import random

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if __name__=='__main__':
    print('Using device:', device)

Using device: cpu


# CS 447 Homework 2 $-$ Text Clasification with Neural Networks
In this homework, you will build machine learning models to detect the sentiment of movie reviews using the IMDb movie reviews dataset. Specifically, you will implement classifiers based on Convolutional Neural Networks (CNN's) and Recurrent Neural Networks (RNN's).

We highly recommend that you take a look at the PyTorch tutorials before starting this assignment:
<ul>
<li>https://pytorch.org/tutorials/beginner/pytorch_with_examples.html
<li>https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
<li>https://github.com/yunjey/pytorch-tutorial
</ul>

We suggest that you select "GPU" as your runtime type, as this will speed up the training of your models. You can find this by going to <TT>Runtime > Change Runtime Type</TT> and select "GPU" from the dropdown menu.

# Step 1: Download the Data
First we will download the dataset using [torchtext](https://torchtext.readthedocs.io/en/latest/index.html), which is a package that supports NLP for PyTorch. The following cell will get you `train_data` and `test_data`. It also does some basic tokenization.

*   To access the list of textual tokens for the *i*th example, use `train_data[i][1]`
*   To access the label for the *i*th example, use `train_data[i][0]`



In [ ]:
### DO NOT EDIT ###

def preprocess(review):
    '''
    Simple preprocessing function.
    '''
    res = []
    for x in review.split(' '):
        remove_beg=True if x[0] in {'(', '"', "'"} else False
        remove_end=True if x[-1] in {'.', ',', ';', ':', '?', '!', '"', "'", ')'} else False
        if remove_beg and remove_end: res += [x[0], x[1:-1], x[-1]]
        elif remove_beg: res += [x[0], x[1:]]
        elif remove_end: res += [x[:-1], x[-1]]
        else: res += [x]
    return res

if __name__=='__main__':
    train_data = torchtext.datasets.IMDB(root='.data', split='train')
    train_data = list(train_data)
    train_data = [(x[0], preprocess(x[1])) for x in train_data]
    train_data, test_data = train_data[0:10000] + train_data[12500:12500+10000], train_data[10000:12500] + train_data[12500+10000:], 

    print('Num. Train Examples:', len(train_data))
    print('Num. Test Examples:', len(test_data))

    print("\nSAMPLE DATA:")
    for x in random.sample(train_data, 5):
        print('Sample text:', x[1])
        print('Sample label:', x[0], '\n')

aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:02<00:00, 28.8MB/s]


Num. Train Examples: 20000
Num. Test Examples: 5000

SAMPLE DATA:
Sample text: ['While', 'I', 'recently', 'gave', 'OPERATION', 'PETTICOAT', 'a', 'positive', 'review', ',', 'I', 'really', "didn't", 'like', 'this', 'film', 'even', 'though', 'it', 'had', 'so', 'many', 'similarities', '.', 'Both', 'were', 'made', 'just', 'a', 'few', 'years', 'apart', ',', 'both', 'starred', 'Cary', 'Grant', 'and', 'both', 'were', 'WWII', 'comedies', '.', 'However', ',', 'the', 'overall', 'tone', 'and', 'style', 'of', 'the', 'films', 'were', 'quite', 'different', '.', 'KISS', 'HER', 'FOR', 'ME', ',', 'overall', ',', 'just', 'seemed', 'like', 'a', 'cheaper', 'film--with', 'poor', 'writing', ',', 'little', 'energy', 'and', 'some', 'VERY', 'broad', 'performances--even', 'when', 'compared', 'to', 'OPERATION', 'PETTICOAT', '.', 'I', 'think', 'that', 'at', 'least', 'much', 'of', 'the', 'blame', 'for', 'this', 'lies', 'in', 'casting', 'Jayne', 'Mansfield', '.', 'The', 'combination', 'of', 'her', 'ample', 'talents'

# Step 2: Create Dataloader [20 points]




## <font color='red'>TODO:</font> Define the Dataset Class [20 Points]

In the following cell, we will define the <b>dataset</b> class. The dataset contains the tokenized data for your model. You need to implement the following functions: 

*   <b>` build_dictionary(self)`:</b>  <b>[10 points]</b> Creates the dictionaries `idx2word` and `word2idx`. You will represent each word in the dataset with a unique index, and keep track of this in these dictionaries. Use the hyperparameter `threshold` to control which words appear in the dictionary: a training word’s frequency should be `>= threshold` to be included in the dictionary.

* <b>`convert_text(self)`:</b> Converts each review in the dataset to a list of indices, given by your `word2idx` dictionary. You should store this in the `textual_ids` variable, and the function does not return anything. If a word is not present in the  `word2idx` dictionary, you should use the `<UNK>` token for that word. Be sure to append the `<END>` token to the end of each review.

*   <b>` get_text(self, idx) `:</b> Return the review at `idx` in the dataset as an array of indices corresponding to the words in the review. If the length of the review is less than `max_len`, you should pad the review with the `<PAD>` character up to the length of `max_len`. If the length is greater than `max_len`, then it should only return the first `max_len` words. The return type should be `torch.LongTensor`.

*   <b>`get_label(self, idx) `</b>: Return the value `1` if the label for `idx` in the dataset is `positive`, and should return `0` if it is `negative`. The return type should be `torch.LongTensor`.

*  <b> ` __len__(self) `:</b> Return the total number of reviews in the dataset as an `int`.

*   <b>` __getitem__(self, idx)`:</b> <b>[10 points]</b> Return the (padded) text, and the label. The return type for both these items should be `torch.LongTensor`. You should use the ` get_label(self, idx) ` and ` get_text(self, idx) ` functions here.


<b>Note:</b> You should convert all words to lower case in your functions.

<b>Autograder Hint:</b> Make sure that you use instance variables such as `self.threshold` throughout your code, rather than the global variable `THRESHOLD` (defined later on). The variable `THRESHOLD` will not be known to the autograder, and the use of it within the class will cause an autograder error.

In [ ]:
PAD = '<PAD>'
END = '<END>'
UNK = '<UNK>'

class TextDataset(data.Dataset):
    def __init__(self, examples, split, threshold, max_len, idx2word=None, word2idx=None):

        self.examples = examples #this is the train data i guess
        assert split in {'train', 'val', 'test'}
        self.split = split #split is 'train' doesn't really matter here
        self.threshold = threshold # threshold is 10
        self.max_len = max_len #max length is 150

        # Dictionaries
        self.idx2word = idx2word #currently it's None
        self.word2idx = word2idx #currently it's None
        if split == 'train':
            self.build_dictionary() #calls the build_dictionary function
        self.vocab_size = len(self.word2idx) #How is this 3?
        
        # Convert text to indices
        self.textual_ids = [] #sorry what is this?
        self.convert_text()

    
    def build_dictionary(self): 
        '''
        Build the dictionaries idx2word and word2idx. This is only called when split='train', as these
        dictionaries are passed in to the __init__(...) function otherwise. Be sure to use self.threshold
        to control which words are assigned indices in the dictionaries.
        Returns nothing.
        '''
        assert self.split == 'train'
        
        # Don't change this
        self.idx2word = {0:PAD, 1:END, 2: UNK}
        self.word2idx = {PAD:0, END:1, UNK: 2}

        ##### TODO #####
        # Count the frequencies of all words in the training data (self.examples) [ ["label",['word','word2','word3',....]] , ["label2",['wordzzz','zzz',....]] ]
          # To access the list of textual tokens for the ith example, use train_data[i][1]
          # To access the label for the ith example, use train_data[i][0]
        length = len(self.examples)
        # print(len(self.examples))
        frequency = {}
        # print(self.examples[0][1])
        for i in range(length):
          for word in self.examples[i][1]:
            word = word.lower()
            if word in frequency:
              frequency[word] += 1
            else:
              frequency[word] = 1
        i = 3
        for word,freq in frequency.items():
          if freq >= self.threshold:
            self.word2idx[word] = i
            i += 1
        # self.word2idx = {key:val for key, val in frequency.items() if val >= self.threshold or key == PAD or key == END or key == UNK}
        # print(self.word2idx)
        # self.idx2word = dict([(value, key) for key, value in self.word2idx.items()])
        self.idx2word = {key:val for val,key in self.word2idx.items()}
        # print(self.idx2word)
        # Assign idx (starting from 3) to all words having word_freq >= self.threshold
        # Make sure you call word.lower() on each word to convert it to lowercase
    
    def convert_text(self):
        '''
        Convert each review in the dataset (self.examples) to a list of indices, given by self.word2idx.
        Store this in self.textual_ids; returns nothing.
        '''
        #so would it look like this? [ [200,322,4444,4,135, ... max length], [0,1,23213,31,423 ... max length], ....]
        # print(self.word2idx['i'], self.word2idx['rented'],self.word2idx[UNK])
        length = len(self.examples)
        # indices_of_one_review = []
        for i in range(length):
          indices_of_one_review = []
          for review in self.examples[i][1]:
            review = review.lower()
            if review in self.word2idx:
              indices_of_one_review.append(self.word2idx[review])
            else:
              indices_of_one_review.append(self.word2idx[UNK])
          indices_of_one_review.append(self.word2idx[END])
          self.textual_ids.append(indices_of_one_review)
        # for i in range(20):
        #   print(self.textual_ids[0][i])
        # print("New one")
        # for i in range(20):
        #   print(self.textual_ids[1][i])
            

        ##### TODO #####
        # Remember to replace a word with the <UNK> token if it does not exist in the word2idx dictionary.
        # Remember to append the <END> token to the end of each review.
        pass

    def get_text(self, idx):
        '''
        Return the review at idx as a long tensor (torch.LongTensor) of integers corresponding to the words in the review.
        You may need to pad as necessary (see above).
        '''
        #Return the review at idx in the dataset as an array of indices corresponding to the words in the review. 
        #If the length of the review is less than max_len, you should pad the review with the <PAD> character up to the length of max_len. 
        #If the length is greater than max_len, then it should only return the first max_len words.
        # query through the textual_ids  [4, 5, '<UNK>', 7, 8, 9, 10,'<END>','<PAD>','<PAD>']  [4, 5, '<UNK>', 7, 8] 
        # the parameter is showing that idx is the index of the review of the textual_ids
        # query first then pytorch at the end
        ##### TODO #####
        length = len(self.textual_ids[idx])
        data = []
        for i in range(self.max_len):
          if i >= length:
            data.append(0)
          else:
            data.append(self.textual_ids[idx][i])
        data = torch.LongTensor(data)
        return data
    
    def get_label(self, idx):
        '''
        This function should return the value 1 if the label for idx in the dataset is 'positive', 
        and 0 if it is 'negative'. The return type should be torch.LongTensor.
        '''
        ##### TODO #####
        # the parameter is showing that idx is the index of the review of the self.examples, and then if it's positive, reutnr
        # querying through self.examples. [0] or [1] just return torch.tensor(0) or torch.tensor(1)
        # To access the label for the ith example, use train_data[i][0]
        hi = torch.LongTensor([1])
        # hi = torch.LongTensor(1)
        # return hi tensor([94688258809728])
        # print(self.examples[idx][0])
        if self.examples[idx][0] == 'pos':
          hi = torch.LongTensor([1])
        if self.examples[idx][0] == 'neg':
          hi = torch.LongTensor([0])
        return torch.squeeze(hi) #tensor(94688258801856)

    def __len__(self):
        '''
        Return the number of reviews (int value) in the dataset
        '''
        ##### TODO #####
        return len(self.examples)
    
    def __getitem__(self, idx):
        '''
        Return the review, and label of the review specified by idx.
        '''
        # Return the (padded) text, and the label. The return type for both these items should be torch.LongTensor. 
        # You should use the get_label(self, idx) and get_text(self, idx) functions here.
        ##### TODO #####
        review = self.get_text(idx)
        label = self.get_label(idx)
        return review, label

In [ ]:
if __name__=='__main__':
    # Sample item
    Ds = TextDataset(train_data, 'train', threshold=10, max_len=150)
    print("Ds", Ds)
    print('Vocab size:', Ds.vocab_size)
    # print(len(Ds)) it's 20,000 but for some reason it keeps printing the same one??
    text, label = Ds[random.randint(0, len(Ds))]
    # text, label = Ds[1]
    print('Example text:', text)
    print('Example label:', label)

Ds <__main__.TextDataset object at 0x7f04246aaf50>
Vocab size: 19002
Example text: tensor([ 1033,     2,     2,    22,  3306,   809,    40,     2,    28,    41,
         5514, 16161,    24,    68,    55,     2, 10801,   105,    13,   154,
           74, 11371,  1710,    74, 12217,    24,    88,   952,    38,     2,
            2,    38,    13, 11371,  8040,   148,   553,   251,     2,   638,
           91,  9555,    74,    34, 10067,  3525,    24,     2,    91,  3402,
           38, 18382,   681,    34,   384,   200,  4229,  2936,    80,    74,
        11371,    38,  6510,    13,  2707,     2,   325,    11,    13,  7931,
           91, 12112,   658,  7944,  3541,    34,     2, 10828,    22,   148,
         1029,    91,  8661,    24,  8535,  5683,    91, 12177,  1555,    38,
         1346,   721,    34,  2084,  1555,    34,     2,     2,     2,    91,
            2,  7387,   179,    18,   209,   384,   638,    24,   258,    55,
          631,    41,  5195,    11,  4874,    38,   180,   

# Step 3: Train a Convolutional Neural Network (CNN) [40 points]

## <font color='red'>TODO:</font> Define the CNN Model [20 points]
Here you will define your convolutional neural network for text classification. We provide you with the CNN class, you need to fill in parts of the `__init__(...)` and `forward(...)` functions. Each of these functions is worth 10 points.

We have provided you with instructions and hints in the comments. In particular, pay attention to the desired shapes; you may find it helpful to print the shape of the tensors as you code. It may also help to keep PyTorch documentation open for the modules & functions you are using, since they describe input and output dimensions.

In [ ]:
class CNN(nn.Module):
    def __init__(self, vocab_size, embed_size, out_channels, filter_heights, stride, dropout, num_classes, pad_idx):
        super(CNN, self).__init__()
        
        ##### TODO #####
        # Create an embedding layer (https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html)
        #   to represent the words in your vocabulary. Make sure to use vocab_size, embed_size, and pad_idx here.
        """
        CNN(vocab_size = train_Ds.vocab_size, # Don't change this
                embed_size = 128, 
                out_channels = 64, 
                filter_heights = [2, 3, 4], 
                stride = 1, 
                dropout = 0.5, 
                num_classes = 2, # Don't change this
                pad_idx = train_Ds.word2idx[PAD]) # Don't change this
        """
        self.embedding = nn.Embedding(vocab_size, embedding_dim=embed_size, padding_idx = pad_idx)

        # Define multiple Convolution layers (nn.Conv2d) with filter (kernel) size [filter_height, embed_size] based on your 
        #   different filter_heights.
        # Input channels will be 1 and output channels will be out_channels (these many different filters will be trained 
        #   for each convolution layer)
        # If you want, you can store a list of modules inside nn.ModuleList.
        # Note: even though your conv layers are nn.Conv2d, we are doing a 1d convolution since we are only moving the filter 
        #   in one direction
        self.convs = nn.ModuleList([nn.Conv2d(in_channels = 1,  out_channels=out_channels, 
                                              kernel_size= (fs,embed_size)) for fs in filter_heights])
        # Create a dropout layer (nn.Dropout) using dropout
        self.dropout = nn.Dropout(p=dropout)

        # Define a linear layer (nn.Linear) that consists of num_classes units 
        #   and takes as input the concatenated output for all cnn layers (out_channels * num_of_cnn_layers units)
        self.fc = nn.Linear(len(filter_heights)*out_channels, num_classes)


    def forward(self, texts):
        """
        texts: LongTensor [batch_size, max_len]
        
        Returns output: Tensor [batch_size, num_classes]
        """
        ##### TODO #####

        # Pass texts through your embedding layer to convert from word ids to word embeddings
        #   Resulting: shape: [batch_size, max_len, embed_size]
        embedded = self.embedding(texts)

        # Input to conv should have 1 channel. Take a look at torch's unsqueeze() function
        #   Resulting shape: [batch_size, 1, MAX_LEN, embed_size]
        embedded = torch.unsqueeze(embedded, 1)
        
        # Pass these texts to each of your conv layers and compute their output as follows:
        #   Your cnn output will have shape [batch_size, out_channels, *, 1] where * depends on filter_height and stride
        #   Convert to shape [batch_size, out_channels, *] (see torch's squeeze() function)
        #   Apply non-linearity on it (F.relu() is a commonly used one. Feel free to try others)
        #   Take the max value across last dimension to have shape [batch_size, out_channels]
        # Concatenate (torch.cat) outputs from all your cnns [batch_size, (out_channels*num_of_cnn_layers)]
        #
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        concat = torch.cat(pooled,dim=1)

        # Let's understand what you just did:
        #   Since each cnn is of different filter_height, it will look at different number of words at a time
        #     So, a filter_height of 3 means your cnn looks at 3 words (3-grams) at a time and tries to extract some information from it
        #   Each cnn will learn out_channels number of features from the words it sees at a time
        #   Then you applied a non-linearity and took the max value for all channels
        #     You are essentially trying to find important n-grams from the entire text
        # Everything happens on a batch simultaneously hence you have that additional batch_size as the first dimension
        
        # Apply dropout
        cat = self.dropout(concat)

        # Pass your output through the linear layer and return its output 
        #   Resulting shape: [batch_size, num_classes]
        

        ##### NOTE: Do not apply a sigmoid or softmax to the final output - done in training method!

        return self.fc(cat)

## Train CNN Model

First, we initialize the train and test <b>dataloaders</b>. A dataloader is responsible for providing batches of data to your model. Notice how we first instantiate datasets for the train and test data, and that we use the training vocabulary for both.

You do not need to edit this cell.

In [ ]:
if __name__=='__main__':
    THRESHOLD = 5 # Don't change this
    MAX_LEN = 100 # Don't change this
    BATCH_SIZE = 32 # Feel free to try other batch sizes

    train_Ds = TextDataset(train_data, 'train', THRESHOLD, MAX_LEN)
    train_loader = torch.utils.data.DataLoader(train_Ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, drop_last=True)

    test_Ds = TextDataset(test_data, 'test', THRESHOLD, MAX_LEN, train_Ds.idx2word, train_Ds.word2idx)
    test_loader = torch.utils.data.DataLoader(test_Ds, batch_size=1, shuffle=False, num_workers=1, drop_last=False)

Now we provide you with a function that takes your model and trains it on the data.

You do not need to edit this cell. However, you may want to write code to save your model periodically, as Colab connections are not permanent. See the tutorial here if you wish to do this: https://pytorch.org/tutorials/beginner/saving_loading_models.html.

In [ ]:
### DO NOT EDIT ###

from tqdm.notebook import tqdm

def train_model(model, num_epochs, data_loader, optimizer, criterion):
    print('Training Model...')
    model.train()
    for epoch in tqdm(range(num_epochs)):
        epoch_loss = 0
        epoch_acc = 0
        for texts, labels in data_loader:
            texts = texts.to(device) # shape: [batch_size, MAX_LEN]
            labels = labels.to(device) # shape: [batch_size]

            optimizer.zero_grad()

            output = model(texts)
            acc = accuracy(output, labels)
            
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        print('[TRAIN]\t Epoch: {:2d}\t Loss: {:.4f}\t Train Accuracy: {:.2f}%'.format(epoch+1, epoch_loss/len(data_loader), 100*epoch_acc/len(data_loader)))
    print('Model Trained!\n')

Here are some other helper functions we will need.

In [ ]:
### DO NOT EDIT ###

def count_parameters(model):
    """
    Count number of trainable parameters in the model
    """
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


def accuracy(output, labels):
    """
    Returns accuracy per batch
    output: Tensor [batch_size, n_classes]
    labels: LongTensor [batch_size]
    """
    preds = output.argmax(dim=1) # find predicted class
    correct = (preds == labels).sum().float() # convert into float for division 
    acc = correct / len(labels)
    return acc

Now you can instantiate your model. We provide you with some recommended hyperparameters; you should be able to get the desired accuracy with these, but feel free to play around with them.

In [ ]:
if __name__=='__main__':
    cnn_model = CNN(vocab_size = train_Ds.vocab_size, # Don't change this
                embed_size = 128, 
                out_channels = 64, 
                filter_heights = [2, 3, 4], 
                stride = 1, 
                dropout = 0.5, 
                num_classes = 2, # Don't change this
                pad_idx = train_Ds.word2idx[PAD]) # Don't change this

    # Put your model on the device (cuda or cpu)
    cnn_model = cnn_model.to(device)
    
    print('The model has {:,d} trainable parameters'.format(count_parameters(cnn_model)))

The model has 3,879,746 trainable parameters


Next, we create the **criterion**, which is our loss function: it is a measure of how well the model matches the empirical distribution of the data. We use cross-entropy loss (https://en.wikipedia.org/wiki/Cross_entropy).

We also define the **optimizer**, which performs gradient descent. We use the Adam optimizer (https://arxiv.org/pdf/1412.6980.pdf), which has been shown to work well on these types of models.

In [ ]:
if __name__=='__main__':    
    LEARNING_RATE = 5e-4 # Feel free to try other learning rates

    # Define the loss function
    criterion = nn.CrossEntropyLoss().to(device)

    # Define the optimizer
    optimizer = optim.Adam(cnn_model.parameters(), lr=LEARNING_RATE)

NameError: ignored

Finally, we can train the model.

In [ ]:
if __name__=='__main__':    
    N_EPOCHS = 20 # Feel free to change this
    
    # train model for N_EPOCHS epochs
    train_model(cnn_model, N_EPOCHS, train_loader, optimizer, criterion)

Training Model...


  0%|          | 0/20 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)


[TRAIN]	 Epoch:  1	 Loss: 0.6981	 Train Accuracy: 58.42%
[TRAIN]	 Epoch:  2	 Loss: 0.5865	 Train Accuracy: 68.44%
[TRAIN]	 Epoch:  3	 Loss: 0.5446	 Train Accuracy: 72.18%
[TRAIN]	 Epoch:  4	 Loss: 0.5006	 Train Accuracy: 75.36%
[TRAIN]	 Epoch:  5	 Loss: 0.4551	 Train Accuracy: 78.40%
[TRAIN]	 Epoch:  6	 Loss: 0.4130	 Train Accuracy: 80.87%
[TRAIN]	 Epoch:  7	 Loss: 0.3711	 Train Accuracy: 83.46%
[TRAIN]	 Epoch:  8	 Loss: 0.3245	 Train Accuracy: 85.84%
[TRAIN]	 Epoch:  9	 Loss: 0.2836	 Train Accuracy: 87.97%
[TRAIN]	 Epoch: 10	 Loss: 0.2408	 Train Accuracy: 89.96%
[TRAIN]	 Epoch: 11	 Loss: 0.2030	 Train Accuracy: 91.71%
[TRAIN]	 Epoch: 12	 Loss: 0.1680	 Train Accuracy: 93.39%
[TRAIN]	 Epoch: 13	 Loss: 0.1372	 Train Accuracy: 94.66%
[TRAIN]	 Epoch: 14	 Loss: 0.1181	 Train Accuracy: 95.50%
[TRAIN]	 Epoch: 15	 Loss: 0.0877	 Train Accuracy: 96.77%
[TRAIN]	 Epoch: 16	 Loss: 0.0836	 Train Accuracy: 96.98%
[TRAIN]	 Epoch: 17	 Loss: 0.0649	 Train Accuracy: 97.59%
[TRAIN]	 Epoch: 18	 Loss: 0.054

## Evaluate CNN Model [20 points]

Now that we have trained a model for text classification, it is time to evaluate it. We have provided you with a function to do this; you do not need to modify anything.

To pass the autograder for the CNN, you will need to achieve **73% accuracy** on the test set. Note that Gradescope uses a different test set; however, it is very similar, and the accuracies between the two datasets should be comparable.

In [ ]:
### DO NOT EDIT ###

import random

def evaluate(model, data_loader, criterion):
    print('Evaluating performance on the test dataset...')
    model.eval()
    epoch_loss = 0
    epoch_acc = 0
    all_predictions = []
    print("\nSOME PREDICTIONS FROM THE MODEL:")
    for texts, labels in tqdm(data_loader):
        texts = texts.to(device)
        labels = labels.to(device)
        
        output = model(texts)
        acc = accuracy(output, labels)
        pred = output.argmax(dim=1)
        all_predictions.append(pred)
        
        loss = criterion(output, labels)
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()

        if random.random() < 0.0015:
            print("Input: "+' '.join([data_loader.dataset.idx2word[idx] for idx in texts[0].tolist() if idx not in {data_loader.dataset.word2idx[PAD], data_loader.dataset.word2idx[END]}]))
            print("Prediction:", pred.item(), '\tCorrect Output:', labels.item(), '\n')

    full_acc = 100*epoch_acc/len(data_loader)
    full_loss = epoch_loss/len(data_loader)
    print('[TEST]\t Loss: {:.4f}\t Accuracy: {:.2f}%'.format(full_loss, full_acc))
    predictions = torch.cat(all_predictions)
    return predictions, full_acc, full_loss

In [ ]:
if __name__=='__main__':
    evaluate(cnn_model, test_loader, criterion) # Compute test data accuracy

Evaluating performance on the test dataset...

SOME PREDICTIONS FROM THE MODEL:


  0%|          | 0/5000 [00:00<?, ?it/s]

Input: why do all movies on lifetime have such anemic titles ? " an unexpected love " - ooh , how <UNK> ! " this much i know " would have been better . the film is nothing special . real people don't really talk like these characters do and the situations are really hackneyed . the straight woman who " turns " lesbian seemed more butch than the lesbian character . if you wanna watch two hot women kiss in a very discreet fashion , you might enjoy this . although it seems like it was written by someone who
Prediction: 0 	Correct Output: 0 

Input: all internet buzz aside , this movie was god awful . i expected the movie to be more of a farce than anything . instead the film makers tried to make a serious thriller/horror movie , and they completely missed . there were only a few good parts , and a couple good lines by samuel jackson . other than that , it was a bunch of gore and some poorly animated snakes . all of the internet joking was miles better than the actual movie . now that the 

# Step 4: Train a Recurrent Neural Network (RNN) [40 points]
You will now build a text clasification model that is based on **recurrences**.

## <font color='red'>TODO:</font> Define the RNN Model [20 points]

First, you will define the RNN. As with the CNN, we provide you with the skeleton of the class, and you need to fill in parts of the `__init__(...)` and `forward(...)` methods. Each of these functions is worth 10 points.

In [ ]:
class RNN(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers, bidirectional, dropout, num_classes, pad_idx):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.bidirectional = bidirectional

        ##### TODO #####
        # Create an embedding layer (https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html)
        #   to represent the words in your vocabulary. Make sure to use vocab_size, embed_size, and pad_idx here.
        self.embedding = nn.Embedding(vocab_size, embed_size, padding_idx = pad_idx)

        # Create a recurrent network (use nn.GRU, not nn.LSTM) with batch_first = True
        # Make sure you use hidden_size, num_layers, dropout, and bidirectional here.
        self.rnn = nn.GRU(embed_size, hidden_size, num_layers = num_layers, bidirectional = bidirectional, dropout = dropout, batch_first = True)
        
        # Create a dropout layer (nn.Dropout) using dropout
        self.dropout = nn.Dropout(dropout)

        # Define a linear layer (nn.Linear) that consists of num_classes units 
        #   and takes as input the output of the last timestep. In the bidirectional case, you should concatenate
        #   the output of the last timestep of the forward direction with the output of the last timestep of the backward direction).
        if self.bidirectional:
          self.fc = nn.Linear(hidden_size*2, num_classes)
        else:
          self.fc = nn.Linear(hidden_size, num_classes)


    def forward(self, texts):
        """
        texts: LongTensor [batch_size, MAX_LEN]
        
        Returns output: Tensor [batch_size, num_classes]
        """
        ##### TODO #####
        """
        RNN(vocab_size = train_Ds.vocab_size, # Don't change this
                embed_size = 128, 
                hidden_size = 128, 
                num_layers = 2,
                bidirectional = True,
                dropout = 0.5,
                num_classes = 2, # Don't change this
                pad_idx = train_Ds.word2idx[PAD]) # Don't change this
        """
        # Pass texts through your embedding layer to convert from word ids to word embeddings
        #   Resulting: shape: [batch_size, max_len, embed_size]
        # texts [batch_size, MAX_LEN] is torch.Size([32, 100])
        # embedding (vocabsize = 29730, 128, ...)
        embedded = self.embedding(texts) #torch.Size([32, 100, 128])

        # Pass the result through your recurrent network
        #   See PyTorch documentation for resulting shape for nn.GRU
        output, hidden = self.rnn(embedded)#, 2 if self.bidirectional else 1)
        # output is torch.Size([4, 32, 128])
        # hidden is torch.Size([32, 100, 256])

        #hidden is the final hidden state for each element in the batch
        # output features (h_t) from the last layer of the GRU, for each t.
        # output, output_lengths = nn.utils.rnn.pad_packed_sequence(output)

        # Concatenate the outputs of the last timestep for each direction (see torch.cat(...))
        #   This depends on whether or not your model is bidirectional.
        #   Resulting shape: [batch_size, num_dirs*hidden_size]
        # concatenate from the last hidden GRU. layer which is the output o.. hidden is already the last timestep of GRU
        # extract last timestep for each direction if bidirectional, concatenate, else don't
        if self.bidirectional:
          dropped = self.dropout(torch.cat((hidden[-2,:,:],hidden[-1,:,:]), dim = -1))
          #concat = torch.cat([hidden[-1], hidden[-2], dim = -1])
        else:
          dropped = self.dropout(hidden[-1,:,:])
          #dropped = self.dropout(hidden[-1])
        # Apply dropout
        dropped = self.dropout(dropped)


        # # Pass your output through the linear layer and return its output 
        # #   Resulting shape: [batch_size, num_classes]
        prediction = self.fc(dropped)

        ##### NOTE: Do not apply a sigmoid or softmax to the final output - done in training method!
        
        return prediction

## Train RNN Model
First, we initialize the train and test dataloaders.

In [ ]:
if __name__=='__main__':
    THRESHOLD = 5 # Don't change this
    MAX_LEN = 100 # Don't change this
    BATCH_SIZE = 32 # Feel free to try other batch sizes

    train_Ds = TextDataset(train_data, 'train', THRESHOLD, MAX_LEN)
    train_loader = torch.utils.data.DataLoader(train_Ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, drop_last=True)

    test_Ds = TextDataset(test_data, 'test', THRESHOLD, MAX_LEN, train_Ds.idx2word, train_Ds.word2idx)
    test_loader = torch.utils.data.DataLoader(test_Ds, batch_size=1, shuffle=False, num_workers=1, drop_last=False)

Now you can instantiate your model. We provide you with some recommended hyperparameters; you should be able to get the desired accuracy with these, but feel free to play around with them.

In [ ]:
if __name__=='__main__':
    rnn_model = RNN(vocab_size = train_Ds.vocab_size, # Don't change this
                embed_size = 128, 
                hidden_size = 128, 
                num_layers = 2,
                bidirectional = True,
                dropout = 0.5,
                num_classes = 2, # Don't change this
                pad_idx = train_Ds.word2idx[PAD]) # Don't change this

    # Put your model on device
    rnn_model = rnn_model.to(device)

    print('The model has {:,d} trainable parameters'.format(count_parameters(rnn_model)))

The model has 4,300,546 trainable parameters


Here, we create the criterion and optimizer; as with the CNN, we use cross-entropy loss and Adam optimization.

In [ ]:
if __name__=='__main__':    
    LEARNING_RATE = 5e-4 # Feel free to try other learning rates

    # Define your loss function
    criterion = nn.CrossEntropyLoss().to(device)

    # Define your optimizer
    optimizer = optim.Adam(rnn_model.parameters(), lr=LEARNING_RATE)

Finally, we can train the model. We use the same `train_model(...)` function that we defined for the CNN.

In [ ]:
if __name__=='__main__':    
    N_EPOCHS = 15 # Feel free to change this
    
    # train model for N_EPOCHS epochs
    train_model(rnn_model, N_EPOCHS, train_loader, optimizer, criterion)

Training Model...


  0%|          | 0/15 [00:00<?, ?it/s]

[TRAIN]	 Epoch:  1	 Loss: 0.6705	 Train Accuracy: 58.58%
[TRAIN]	 Epoch:  2	 Loss: 0.5492	 Train Accuracy: 72.52%
[TRAIN]	 Epoch:  3	 Loss: 0.4298	 Train Accuracy: 80.80%
[TRAIN]	 Epoch:  4	 Loss: 0.3417	 Train Accuracy: 85.76%
[TRAIN]	 Epoch:  5	 Loss: 0.2666	 Train Accuracy: 89.56%
[TRAIN]	 Epoch:  6	 Loss: 0.2010	 Train Accuracy: 92.50%
[TRAIN]	 Epoch:  7	 Loss: 0.1421	 Train Accuracy: 95.03%
[TRAIN]	 Epoch:  8	 Loss: 0.0958	 Train Accuracy: 96.84%
[TRAIN]	 Epoch:  9	 Loss: 0.0581	 Train Accuracy: 98.14%
[TRAIN]	 Epoch: 10	 Loss: 0.0473	 Train Accuracy: 98.40%
[TRAIN]	 Epoch: 11	 Loss: 0.0356	 Train Accuracy: 98.88%
[TRAIN]	 Epoch: 12	 Loss: 0.0275	 Train Accuracy: 99.07%
[TRAIN]	 Epoch: 13	 Loss: 0.0227	 Train Accuracy: 99.23%
[TRAIN]	 Epoch: 14	 Loss: 0.0219	 Train Accuracy: 99.30%
[TRAIN]	 Epoch: 15	 Loss: 0.0134	 Train Accuracy: 99.56%
Model Trained!



## Evaluate RNN Model [20 points]

Now we can evaluate the RNN. 

To pass the autograder for the RNN, you will need to achieve **75% accuracy** on the test set. Note that Gradescope uses a different test set; however, it is very similar, and the accuracies between the two datasets should be comparable.

In [ ]:
if __name__=='__main__':    
    evaluate(rnn_model, test_loader, criterion) # Compute test data accuracy

Evaluating performance on the test dataset...

SOME PREDICTIONS FROM THE MODEL:


  0%|          | 0/5000 [00:00<?, ?it/s]

Input: an okay film , with a fine leading lady , but a terrible leading man . <UNK> jenkins , who plays the husband , is a truly bad actor . joyce <UNK> , on the other hand , is the movie's saving grace . she's the best actor of the <UNK> /><br <UNK> the first comment , by the fellow who heaped praise upon the movie ( and , according to his imdb.com account , has only written one review -- and guess for what movie? ) is obviously a plant . while the movie is nicely shot , it's
Prediction: 0 	Correct Output: 0 

Input: i saw this film last night following a lot of good reviews from many sources . i would like to point out that if your not ready to try and work out continuously who is who and what it all means you will hate this film.<br /><br />i am still struggling to understand the roles of the actors in this film , the film jumps from different stories and does not allow you to really empathise with any of the roles.<br /><br />for the political <UNK> and those interested in corrupti

# What You Need to Submit

To submit the assignment, download this notebook as a <TT>.py</TT> file. You can do this by going to <TT>File > Download > Download .py</TT>. Then rename it to `hwk2.py`.

You will also need to save the `cnn_model` and `rnn_model`. You can run the cell below to do this. After you save the files to your Google Drive, you need to manually download the files to your computer, and then submit them to the autograder.

You will submit the following files to the autograder:
1.   `hwk2.py`, the download of this notebook as a `.py` file (**not** a `.ipynb` file)
1.   `cnn.pt`, the saved version of your `cnn_model`
1.   `rnn.pt`, the saved version of your `rnn_model`

In [ ]:
### DO NOT EDIT ###

if __name__=='__main__':
    from google.colab import drive
    drive.mount('/content/drive')
    print()

    try:
        cnn_model is None
        cnn_exists = True
    except:
        cnn_exists = False

    try:
        rnn_model is None
        rnn_exists = True
    except:
        rnn_exists = False

    if cnn_exists:
        print("Saving CNN model....") 
        torch.save(cnn_model, "drive/My Drive/cnn.pt")
    if rnn_exists:
        print("Saving RNN model....") 
        torch.save(rnn_model, "drive/My Drive/rnn.pt")
    print("Done!")

Mounted at /content/drive

Saving RNN model....
Done!
